<style>
    pre {
        white-space: pre-wrap;
        word-wrap: break-word;
    }
</style>

<div style="display:flex; justify-content:space-around; align-items:center; background-color:#cccccc; padding:5px; border:2px solid #333333;">
    <a href="https://www.um.es/web/estudios/grados/ciencia-ingenieria-datos/" target="_blank">
    <img src="https://www.um.es/documents/1073494/42130150/LogosimboloUMU-positivo.png" alt="UMU" style="height:200px; width:auto;">
    <a href="https://estudios.upct.es/grado/5251/inicio" target="_blank">
    <img src="https://www.upct.es/contenido/universidad/galeria/identidad-2021/logos/logos-upct/marca-upct/marca-principal/horizontal/azul.png" alt="UPCT" style="height:145px; width:auto;">
</div>

# Asignatura: **Deep Learning**

## Titulación: **Grado en Ciencia e Ingeniería de Datos**

## Práctica 3: Transformers
### **Sesión 1/3: Preprocesamiento de secuencias de caracteres**

**Autores**: Antonio Martínez Sánchez, Juan Morales Sánchez, José Luís Sancho Gómez y Juan Antonio Botía Blaya

<div style="page-break-before: always;"></div>

### Contenidos
- [Requisitos](#requisitos)
- [El dataset](#dataset)
- [Codificación](#code)
- [El problema: clasificación de péptidos](#problema)
- [Ejercicios](#ejercicios)

### Requisitos
<a class='anchor' id='requisitos'></a>

Se trabajará con notebooks de [Jupyter](https://jupyter.org/install) con código Python empleando como intérprete la última versión de [Miniconda](https://docs.anaconda.com/miniconda/). Se requiere la preinstalación (se recomienda utilizar [pip](https://pypi.org/project/pip/)) de los siguientes paquetes de Python:

- [Numpy](https://pypi.org/project/numpy/) (computación numérica)
- [Tensorflow](https://www.tensorflow.org/install/pip?hl=es-419#linux) que incluye a Keras (deep learning)
- [Scikit-learn](https://pypi.org/project/scikit-learn/) (machine learning)
- [Matplotlib](https://pypi.org/project/matplotlib/) (visualización de datos)
- [Pandas](https://pypi.org/project/seaborn/) (manipulación de datos tabulados)

### El dataset
<a class='anchor' id='dataset'></a>

Los péptidos son moléculas biológicas formadas por secuencias cortas de aminoácidos, pudiendo llegar hasta aproximadamente los 100 aminoácidos, a partir de esta cantidad a un péptido se le suele denominar proteína. Los aminoácidos (AA) son los ladrillos de los péptidos, que a su vez son los componentes fundamentales de la estructura y maquinaria celular. Las secuencias de AA que determinan la estructura y función de un péptido están codificadas en los genes, las unidades de información almacenadas en el ADN de los seres vivos. Un gen es un fragmento de ADN que contine una secuencia de nucleótidos que es **transcrito** a un fragmento de ARNm (un intermediario que mantiene la información) y que al alcanzar un ribosoma (la fábrica de péptidos) es **trasladado** a una secuencia de AA. Esta secuencia determina como el péptidos deben plegarse sobre si mismos para generar la estructura 3D que le confiere sus propiedades físico-químicas. Por tanto, estudiando la secuencia de AA de un péptido se puede determinar su función y analizar su función biológica.

![ADN_to_AA](figs/ADN_AA_traduccion.jpg)

Los datos a procesar en esta práctica son dos tablas almacenadas en formato CSV con la secuencia de AAs de una serie de péptidos. La primera columna contine el identificador de los péptidos, para nostros no tiene mayor relevancia. La columna 'aa_seq' contiene la secuencia de AAs codificada con caracteres en mayúscula, cada carácter se corresponde con un AA. Finalmente, la columna 'AMP' determina si este péptido tiene algún efecto antimicrobiano. Cada tabla contiene péptidos diferentes, la tabla *non_amp_ampep_cdhit90.csv* contine péptidos sin efecto antimicrobiano y la tabla *veltri_dramp_cdhit_90.csv' contine péptidos con efecto antimicrobiano o clasificados como AMP.

### Codificación
<a class='anchor' id='code'></a>

Las redes neuronales procesan tensores cuyos valores son numéricos. No obstante, en algunos casos los datos de partida son conjuntos o secuencias de caracteres. El ejemplo más claro es el texto, por ejemplo, este párrafo contine una secuencia de caracteres que, si obviamos los signos de puntuación y acentuación, está formado por las letras del abecedario. En el contexto del lenguaje expresado en este párrafo, los carácteres no tienen ningún significado (semántica) de forma aislada. De forma simplista podemos asumir que las unidades semánticas, o **tokens**, del lenguaje natural son las palabras. Puesto que una red neuronal no puede procesar directamente los tokens en forma de palabras, es necesario codificaros previamente empleando valores numéricos. El **vocabulario** de un lenguaje es el conjunto de los diferentes tokens, en caso de los lenguajes naturales puede tener cientos de miles de elementos dificultando su codificación.

El dataset de esta práctica está expresado en un lenguaje más sencillo y menos ambiguo que el lenguaje natural pero también codificable en forma de texto. En este caso, los tokens están claramente definidos y se corresponden con cada carácter de la secuencia de AA de un péptido, de tal modo, que cada carácter identifica unívocamente un AA. Por tanto, el vocabulario del lenguaje de los péptidos se limita a un conjunto reducido de AA representados en un fichero CSV con caracteres individuales (letras mayúsculas). Un péptido se corresponde con una secuencia de AAs (letras) cuya longitud es variable. 

En esta práctica utilizaremos los métodos de codificación más directos:

---

**CODIFICACIÓN.**

- Integer: se construye una tabla con todos los tokens considerados y se le asigna un índice entero a cada token. Luego a cada token de la secuencia se le asigna el índice correspondiente. Como las secuencias tienen tamaños diferentes, se suele configurar para que los vectores de salida tengan el tamaño de la secuencia más larga asignándose un 0 a los valores de relleno (*padding*).

- Multi-hot: la codificación de salida es un vector binario cuyo tamaño es el número de tokens considerados, si un token aparece en la secuencia codificada el vector tendrá un 1 en la posición correspondiente a ese token, si no un 0. Independientemente del tamaño de la secuencia a codificar, la salida será un vector cuyo tamaño se corresponde con la longitud del vocabulario. # |V| TAMAÑO DEL VECTOR PARA CADA TOKEN

- TF-IDF: es una extensión de multi-hot que en lugar de valores binarios asigna la frecuencia de aparición de un token en un conjunto de datos.


---


La librería Keras proporciona la clase [TextVectorization](https://keras.io/api/layers/preprocessing_layers/text/text_vectorization/) que implementa una capa para la codificación de texto.

### El problema: clasificación de péptidos
<a class='anchor' id='problema'></a>

Durante esta práctica abordaremos el problema de la clasificación de fragmentos de texto. En concreto, nos centraremos en la clasificación de péptidos, secuencias de AAs, codificadas mediante caracteres de texto. Los péptidos se pueden agrupar en dos clases según presenten o no actividad antimicrobiana (AMP). Los CSV de entrada están separados entre los AMP y los no AMP, así que deberán fusionarse para construir un dataset que permita un entrenamiento supervisado.

En las diferentes sesiones de esta práctica construiremos diferentes modelos para resolver este problema. El punto de partida será la arquitectura genérica MLP, utilizaremos también redes neuronales recurrentes (RNN) para estudiar la ventaja de procesar datos en forma de secuencia y finalmente implementaremos un Transformer que se ha convertido en el modelo de referencia para este tipo de datos.

### Ejercicios
<a class='anchor' id='ejercicios'></a>

**E1:** Carga las tablas almacenadas en los CSV utilizando la librería Pandas y fusiónalas en un solo Dataframe. Después construye una lista con todos los péptidos y otra cuyos elementos sean booleanos identificando si son AMP. ¿Cuál es la secuencia más larga? ¿Cuántos AAs diferentes hay? ¿Cuál es el vocabulario del lenguaje?

**E2:** Divide los conjuntos de datos del E1 en dos. Uno que contenga el 80% de las secuencias para el entrenamiento supervisado y otro que contenga al resto para testear los modelos entrenados. Asegúrate que en ambos conjuntos hay una selección representativos de péptidos.

**E3:** Codifica los péptidos empleando la codificación multi-hot. ¿Qué longitud tiene cada péptido codificado?

**E4:** Basándote en la arquitectura del E4 de la Sesión 1 de la Práctica 2 (Redes Convolucionales), construye un MLP para clasificar los péptidos. ¿Cuántos parámetros a entrenar tiene?

**E5:** Entrena el modelo del E4 utilizando el 20% de los datos de entrenamiento para validación durante el entrenamiento. Utiliza una cantidad de épocas que produzca sobreajuste, además añade un *callback* para guardar el modelo que tenga una mayor precisión en la validación.

**E6:** Testea la precisión del modelo entrenado en el E5 empleando el conjunto de test obtenido en el E2. Recuerda cargar el modelo para la mejor época. ¿Qué precisión se alcanza?

**E7:** Repite los ejercicios E3-6 pero esta vez empleando codificación TF-IDF. ¿Ha mejorado la precisión? ¿por qué?

**E8:** Hasta ahora hemos trabajado solo con unigramas, los tokens están compuesto por AAs aislados. También es posible construir tokens con combinaciones de AAs (n-gramas). Repite el ejercicio E7 pero esta vez empleando bigramas. ¿Qué tamaño tiene el vocabulario ahora? ¿Por qué MLP ha aumentado considerablemente el número de parámetros a entrenar? ¿Qué ventaja podría aportar el trabajar con n-gramas (n > 1) en comparación de unigramas?

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
# CONJUNTO DE ENTRENAMIENTO DE LA CODIFICACIÓN: TODOS LAS CADENAS DE AMINOACIDOS, UTILIZAR ADAPT Y EMPLEAR PARA CODIFICAR CORRECTAMENTE CADA CARÁCTER DENTRO DE LA CADENA

peptidos_AM = pd.read_csv('../data/dataset_p3/veltri_dramp_cdhit_90.csv', sep=',')

perptidos_sin_efecto_AM =  pd.read_csv('../data/dataset_p3/non_amp_ampep_cdhit90.csv', sep=',')
#ENTRADA: CADENA DE CARÁCTERES Y SALIDA: VECTOR CODIFICADO

In [5]:
peptidos_AM.drop(columns=['Unnamed: 0'], inplace=True)
perptidos_sin_efecto_AM.drop(columns=['Unnamed: 0'], inplace=True)